

> **DATA LOADING**



In [ ]:
import pandas as pd
batting = pd.read_csv("Final_Batting_Matchups.csv")
bowling = pd.read_csv("bowling_matchups.csv")

In [ ]:
batting.columns

Index(['match_id', 'season', 'start_date', 'venue', 'city', 'Player_Team',
       'Opponent', 'batting_innings', 'Batsman', 'batting_position',
       'entry_over', 'exit_over', 'Bowler', 'bowling_style', 'runs',
       'balls_faced', 'Dots', 'One', 'Two', 'Three', 'Four', 'Six', 'SR',
       'toss_winner', 'toss_decision', 'result', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match'],
      dtype='object')

In [ ]:
bowling.columns

Index(['match_id', 'season', 'start_date', 'venue', 'city', 'toss_winner',
       'toss_decision', 'Player_Team', 'Opponent', 'bowling_innings', 'Player',
       'balls_bowled', 'wickets', 'runs_given', 'overs_bowled', 'economy',
       'bowling_average', 'result', 'winner', 'win_by_runs', 'win_by_wickets',
       'player_of_match', 'Bowling Points', 'Bowling Style', 'Batsmen_faced'],
      dtype='object')



> **BATTING PREDICTIONS**



**Data Encoding**

In [ ]:
from sklearn.preprocessing import LabelEncoder

#Creating label encoder objects
venue_encoder = LabelEncoder()
bowling_style_encoder = LabelEncoder()

#Label Encoding the venue and bowling style columns
batting['venue_encoded'] = venue_encoder.fit_transform(batting['venue'])

batting['bowling_style_encoded'] = bowling_style_encoder.fit_transform(batting['bowling_style'])



#Finding all unique teams involved
all_teams = set(batting['Player_Team'].unique()).union(set(batting['Opponent'].unique()))

#Dictionary to map team names to unique integer values
team_encoding = {team: i for i, team in enumerate(all_teams)}

batting['Player_Team_encoded'] = batting['Player_Team'].map(team_encoding)
batting['Opponent_encoded'] = batting['Opponent'].map(team_encoding)



import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, concatenate


if 'Batsman' not in batting.columns or 'Bowler' not in batting.columns:
    raise ValueError("Columns 'Batsman' and 'Bowler' are required.")

#LabelEncoder to encode 'Batsman' and 'Bowler'
label_encoder_batsman = LabelEncoder()
label_encoder_bowler = LabelEncoder()

batting['Batsman_encoded'] = label_encoder_batsman.fit_transform(batting['Batsman'])
batting['Bowler_encoded'] = label_encoder_bowler.fit_transform(batting['Bowler'])

# Embeddings for Batsman and Bowler names
num_batsmen = len(label_encoder_batsman.classes_)
num_bowlers = len(label_encoder_bowler.classes_)
embedding_dim = 10

batsman_input = Input(shape=(1,), name='batsman_input')
batsman_embedding = Embedding(input_dim=num_batsmen, output_dim=embedding_dim, input_length=1)(batsman_input)
batsman_embedding = Flatten()(batsman_embedding)

bowler_input = Input(shape=(1,), name='bowler_input')
bowler_embedding = Embedding(input_dim=num_bowlers, output_dim=embedding_dim, input_length=1)(bowler_input)
bowler_embedding = Flatten()(bowler_embedding)

# Combine embeddings with other features
input_features = ['venue_encoded', 'Player_Team_encoded', 'Opponent_encoded', 'batting_innings', 'batting_position', 'bowling_style_encoded']
X_other = batting[input_features]

# Create a Keras Model to get embeddings
embedding_model = Model(inputs=[batsman_input, bowler_input], outputs=[batsman_embedding, bowler_embedding])

# Get embeddings
batsman_embedding, bowler_embedding = embedding_model.predict([batting['Batsman_encoded'].values.reshape(-1, 1), batting['Bowler_encoded'].values.reshape(-1, 1)])

# Convert embeddings to DataFrames
batsman_df = pd.DataFrame(batsman_embedding, columns=[f'batsman_embedding_{i}' for i in range(embedding_dim)])
bowler_df = pd.DataFrame(bowler_embedding, columns=[f'bowler_embedding_{i}' for i in range(embedding_dim)])

783/783 [==============================] - 1s 2ms/step


**Defining Variables and Splitting Data**

In [ ]:
X = pd.concat([X_other, batsman_df, bowler_df], axis=1)
y_runs = batting['runs']

#Splitting Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_runs, test_size=0.2, random_state=42)

**MODEL TRAINING**

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

y_runs = batting['runs']
y_fours = batting['Four']
y_sixes = batting['Six']
y_balls_faced = batting['balls_faced']

# Split data into training and testing sets
X_train, X_test, y_runs_train, y_runs_test, y_fours_train, y_fours_test, y_sixes_train, y_sixes_test, y_balls_faced_train, y_balls_faced_test = train_test_split(X, y_runs, y_fours, y_sixes, y_balls_faced, test_size=0.2, random_state=42)

# Initializing three separate Gradient Boosting Regressors for each target variable
gb_regressor_runs = GradientBoostingRegressor(n_estimators=150, learning_rate=0.1, max_depth=3, random_state=42)
gb_regressor_fours = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
gb_regressor_sixes = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
gb_regressor_balls_faced = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)


# Train the models
gb_regressor_runs.fit(X_train, y_runs_train)
gb_regressor_fours.fit(X_train, y_fours_train)
gb_regressor_sixes.fit(X_train, y_sixes_train)
gb_regressor_balls_faced.fit(X_train, y_balls_faced_train)


# Predict on the test set for each target variable
y_runs_pred = gb_regressor_runs.predict(X_test)
y_fours_pred = gb_regressor_fours.predict(X_test)
y_sixes_pred = gb_regressor_sixes.predict(X_test)
y_balls_faced_pred = gb_regressor_balls_faced.predict(X_test)


# Evaluate the models (you may want to use different evaluation metrics for each target variable)
mse_runs = mean_squared_error(y_runs_test, y_runs_pred)
mse_fours = mean_squared_error(y_fours_test, y_fours_pred)
mse_sixes = mean_squared_error(y_sixes_test, y_sixes_pred)
mse_balls_faced = mean_squared_error(y_balls_faced_test, y_balls_faced_pred)


print("Mean Squared Error (Runs):", mse_runs)
print("Mean Squared Error (Fours):", mse_fours)
print("Mean Squared Error (Sixes):", mse_sixes)
print("Mean Squared Error (balls_faced):", mse_balls_faced)


Mean Squared Error (Runs): 36.19788943890774
Mean Squared Error (Fours): 0.7220944111519029
Mean Squared Error (Sixes): 0.3541788655308027
Mean Squared Error (balls_faced): 8.431469865776574


**Predicting For A Entire Dataset**

In [ ]:
# Predict on the test set for each target variable
y_runs_pred = gb_regressor_runs.predict(X)
y_fours_pred = gb_regressor_fours.predict(X)
y_sixes_pred = gb_regressor_sixes.predict(X)
y_balls_faced_pred = gb_regressor_balls_faced.predict(X)


# Evaluate the models
mse_runs = mean_squared_error(batting['runs'], y_runs_pred)
mse_fours = mean_squared_error(batting['Four'], y_fours_pred)
mse_sixes = mean_squared_error(batting['Six'], y_sixes_pred)
mse_balls_faced = mean_squared_error(batting['balls_faced'], y_balls_faced_pred)

print("Mean Squared Error (Runs):", mse_runs)
print("Mean Squared Error (Fours):", mse_fours)
print("Mean Squared Error (Sixes):", mse_sixes)
print("Mean Squared Error (balls_faced):", mse_balls_faced)

Mean Squared Error (Runs): 33.90189759522511
Mean Squared Error (Fours): 0.6616969250576957
Mean Squared Error (Sixes): 0.315282003907107
Mean Squared Error (balls_faced): 7.899988066146038


In [ ]:
import numpy as np
y_pred_runs_rounded = np.round(y_runs_pred)
y_pred_fours_rounded = np.round(y_fours_pred)
y_pred_sixes_rounded = np.round(y_sixes_pred)
y_pred_balls_faced_rounded = np.round(y_balls_faced_pred)

**CREATING A NEW DATAFRAME OF THE PREDICTIONS VS ACTUAL VALUES**

In [ ]:
#DataFrame to store the total predicted runs for each batsman in each match
predicted_runs_df = pd.DataFrame(columns=['Match_id', 'Batsman'])

# Group by 'Match_id' and 'Batsman', then sum the predicted runs
predicted_runs_df['Total_Runs_Predicted'] = y_pred_runs_rounded
predicted_runs_df['Fours_Predicted'] = y_pred_fours_rounded
predicted_runs_df['Sixes_Predicted'] = y_pred_sixes_rounded
predicted_runs_df['balls_faced'] = y_pred_balls_faced_rounded
predicted_runs_df['Actual Runs'] = batting['runs']
predicted_runs_df['Actual Fours'] = batting['Four']
predicted_runs_df['Actual Sixes'] = batting['Six']
predicted_runs_df['Actual Balls Faced'] = batting['balls_faced']
predicted_runs_df['Match_id'] = batting['match_id']
predicted_runs_df['PlayerName'] = batting['Batsman']

predicted_runs_df = predicted_runs_df.groupby(['Match_id', 'PlayerName'], as_index=False).sum()

predicted_runs_df['Actual SR'] = (predicted_runs_df['Actual Runs'] / predicted_runs_df['Actual Balls Faced'])*100
predicted_runs_df['SR'] = (predicted_runs_df['Total_Runs_Predicted'] / predicted_runs_df['balls_faced'])*100

predicted_runs_df

,Match_id,PlayerName,Batsman,Total_Runs_Predicted,Fours_Predicted,Sixes_Predicted,balls_faced,Actual Runs,Actual Fours,Actual Sixes,Actual Balls Faced,Actual SR,SR
0,980901,AM Rahane,0,43.0,6.0,0.0,33.0,65,7,3,41,158.536585,130.303030
1,980901,AT Rayudu,0,29.0,1.0,0.0,21.0,22,2,0,27,81.481481,138.095238
2,980901,F du Plessis,0,41.0,5.0,0.0,31.0,34,1,3,33,103.030303,132.258065
3,980901,HH Pandya,0,20.0,3.0,0.0,15.0,9,2,0,12,75.000000,133.333333
4,980901,Harbhajan Singh,0,23.0,1.0,0.0,18.0,45,7,1,30,150.000000,127.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7606,1370353,RD Gaikwad,0,40.0,5.0,0.0,30.0,26,3,1,16,162.500000,133.333333
7607,1370353,Rashid Khan,0,6.0,0.0,0.0,5.0,0,0,0,2,0.000000,120.000000
7608,1370353,S Dube,0,34.0,5.0,0.0,26.0,32,0,2,21,152.380952,130.769231
7609,1370353,Shubman Gill,0,32.0,4.0,0.0,25.0,39,7,0,20,195.000000,128.000000


In [ ]:
predicted_runs_df.columns

Index(['Match_id', 'PlayerName', 'Batsman', 'Total_Runs_Predicted',
       'Fours_Predicted', 'Sixes_Predicted', 'balls_faced', 'Actual Runs',
       'Actual Fours', 'Actual Sixes', 'Actual Balls Faced', 'Actual SR',
       'SR'],
      dtype='object')

**FUNCTION TO CALCULATE BATTING POINTS**

In [ ]:
# Define a function to calculate batting points for actual data
def calculate_batting_points_actual(row):
    runs = row['Actual Runs']
    fours = row['Actual Fours']
    sixes = row['Sixes_Predicted']
    balls_faced = row['Actual Sixes']
    strike_rate = row['Actual SR']

    # Check for NaN values
    if any(pd.isnull(value) for value in [runs, fours, sixes, balls_faced, strike_rate]):
        return 0

    points = runs  # One point for each run

    if fours > 0:
        points += 1 * fours  # 1 bonus point for each four
    if sixes > 0:
        points += 2 * sixes  # 2 bonus points for each six
    if runs >= 50:
        points += 8  # 8 bonus points for 50 or more runs
    if runs >= 100:
        points += 16  # 16 bonus points for 100 or more runs
    if runs == 0:
        points -= 2  # -2 points for getting out at 0 (duck out)
    if balls_faced > 10:
        if 60 <= strike_rate <= 70:
            points -= 2  # -2 if more than 10 balls played and strike rate between 60 to 70
        elif 50 <= strike_rate < 60:
            points -= 4  # -4 if more than 10 balls played and strike rate between 50 to 50.99
        elif strike_rate < 50:
            points -= 6  # -6 if more than 10 balls played and strike rate below 50

    return points


def calculate_batting_points_predicted(row):
    runs = row['Total_Runs_Predicted']
    fours = row['Fours_Predicted']
    sixes = row['Sixes_Predicted']
    balls_faced = row['balls_faced']
    strike_rate = row['SR']

    if any(pd.isnull(value) for value in [runs, fours, sixes, balls_faced, strike_rate]):
        return 0

    points = runs  

    if fours > 0:
        points += 1 * fours 
    if sixes > 0:
        points += 2 * sixes 
    if runs >= 50:
        points += 8  
    if runs >= 100:
        points += 16 
    if runs == 0:
        points -= 2  
    if balls_faced > 10:
        if 60 <= strike_rate <= 70:
            points -= 2  
        elif 50 <= strike_rate < 60:
            points -= 4  
        elif strike_rate < 50:
            points -= 6  

    return points

predicted_runs_df['Predicted Batting Points'] = predicted_runs_df.apply(calculate_batting_points_predicted, axis=1)
predicted_runs_df['Actual Batting Points'] = predicted_runs_df.apply(calculate_batting_points_actual, axis=1)

**SORTING PLAYERS BY HIGHEST POINTS MATCH WISE**

In [ ]:
# first sorting according to predicted points
points_data_sorted = predicted_runs_df.sort_values(by=['Match_id', 'Predicted Batting Points'], ascending=[True, False])
points_data_sorted.to_csv('batting_players_predictions.csv', index=False)

In [ ]:
# now sorting according to actual points
points_data_sorted_actual = predicted_runs_df.sort_values(by=['Match_id', 'Actual Batting Points'], ascending=[True, False])
points_data_sorted_actual.to_csv('batting_players_actual.csv', index=False)



> **BOWLING PREDICTIONS**



**ENCODING DATA**

Similar Preprocessing like the batting data

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create label encoder objects
venue_encoder = LabelEncoder()
bowling_style_encoder = LabelEncoder()

bowling['venue_encoded'] = venue_encoder.fit_transform(bowling['venue'])

bowling['bowling_style_encoded'] = bowling_style_encoder.fit_transform(bowling['Bowling Style'])

all_teams = set(bowling['Player_Team'].unique()).union(set(bowling['Opponent'].unique()))

team_encoding = {team: i for i, team in enumerate(all_teams)}

bowling['Player_Team_encoded'] = bowling['Player_Team'].map(team_encoding)
bowling['Opponent_encoded'] = bowling['Opponent'].map(team_encoding)


import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten

if 'Player' not in bowling.columns:
    raise ValueError("Column 'Bowler' is required.")

label_encoder_bowler = LabelEncoder()
bowling['Bowler_encoded'] = label_encoder_bowler.fit_transform(bowling['Player'])

# Now, create embeddings for Bowler
num_bowlers = len(label_encoder_bowler.classes_)
embedding_dim = 10

bowler_input = Input(shape=(1,), name='bowler_input')
bowler_embedding = Embedding(input_dim=num_bowlers, output_dim=embedding_dim, input_length=1)(bowler_input)
bowler_embedding = Flatten()(bowler_embedding)

# Creating a Keras Model to get embeddings
embedding_model = Model(inputs=bowler_input, outputs=bowler_embedding)

# Get embeddings
bowler_embedding = embedding_model.predict(bowling['Bowler_encoded'].values.reshape(-1, 1))

# Convert embeddings to DataFrames
bowler_df = pd.DataFrame(bowler_embedding, columns=[f'bowler_embedding_{i}' for i in range(embedding_dim)])

174/174 [==============================] - 0s 2ms/step


**VARIABLE DEFINITION AND SPLITTING**

In [ ]:
input_features = ['venue_encoded', 'Player_Team_encoded', 'Opponent_encoded', 'bowling_innings', 'bowling_style_encoded']
X_other = bowling[input_features]

X = pd.concat([X_other, bowler_df], axis=1)
y_wicket = bowling['wickets']
y_economy = bowling['economy']



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score

# Define classes for wickets
def categorize_wickets(num_wickets):
    if num_wickets == 0:
        return '0 wickets'
    elif num_wickets == 1:
        return '1 wickets'
    elif num_wickets == 2:
        return '2 wickets'
    elif num_wickets == 3:
        return '3 wickets'
    elif num_wickets == 4:
        return '4 wickets'
    elif num_wickets == 5:
        return '5 wickets'
    elif num_wickets == 6:
        return '6 wickets'
    elif num_wickets == 7:
        return '7 wickets'
    else:
        return '8+ wickets'

bowling['Wickets_class'] = bowling['wickets'].apply(categorize_wickets)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(bowling['Wickets_class'])
y_wicket = y_encoded

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Assuming 'X' contains your feature columns and 'y_wickets' contains your target variable for wickets

# Split data for wickets prediction
X_train_wickets, X_test_wickets, y_wickets_train, y_wickets_test = train_test_split(X, y_wicket, test_size=0.2, random_state=42)

# Initialize Gradient Boosting Classifier for wickets prediction
gb_classifier = GradientBoostingClassifier()

# Train the model for wickets prediction
gb_classifier.fit(X_train_wickets, y_wickets_train)

# Predict on the test set for wickets prediction
y_wickets_pred = gb_classifier.predict(X_test_wickets)

# Evaluate the model for wickets prediction
accuracy_wickets = accuracy_score(y_wickets_test, y_wickets_pred)
precision_wickets = precision_score(y_wickets_test, y_wickets_pred, average='weighted')
print("Accuracy (Wickets):", accuracy_wickets)
print("Precision (Wickets):", precision_wickets)



from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

# Split data for economy prediction
y_economy = bowling['economy']
X_train_economy, X_test_economy, y_economy_train, y_economy_test = train_test_split(X, y_economy, test_size=0.2, random_state=42)

# Initialize Gradient Boosting Regressor for economy prediction
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train the model for economy prediction
gb_regressor.fit(X_train_economy, y_economy_train)

# Predict on the test set for economy prediction
y_economy_pred = gb_regressor.predict(X_test_economy)

# Evaluate the model for economy prediction
mse_economy = mean_squared_error(y_economy_test, y_economy_pred)
print("Mean Squared Error (Economy):", mse_economy)

Accuracy (Wickets): 0.3900900900900901
Precision (Wickets): 0.3216895643724912
Mean Squared Error (Economy): 8.569882202606234


**PREDICTING FOR ENTIRE DATA**

In [ ]:
y_wickets_pred_all = gb_classifier.predict(X)
y_economy_pred_all = gb_regressor.predict(X)

**CREATING NEW DATAFRAME OF ACTUAL VS PREDICTED**

In [ ]:
# Create a new DataFrame with the predicted values for the entire dataset
prediction_df_all = pd.DataFrame({
    'Match_id': bowling['match_id'],
    'PlayerName': bowling['Player'],
    'Predicted Wickets': label_encoder.inverse_transform(y_wickets_pred_all),
    'Predicted Economy': y_economy_pred_all,
    'Actual Wickets': bowling['wickets'],
    'Actual Economy': bowling['economy'],
    'Actual Bowling Points': bowling['Bowling Points']
})

**BOWLING POINTS**

In [ ]:
# Define a function to calculate bowling points
def calculate_bowling_points(row):
    wickets = row['Predicted Wickets']
    economy = row['Predicted Economy']

    # Check for NaN values
    if any(pd.isnull(value) for value in [wickets, economy]):
        return 0

    if wickets == '0 wickets':
        points = 0
    elif wickets == '1 wickets':
        points = 25
    elif wickets == '2 wickets':
        points = 50
    elif wickets == '3 wickets':
        points = 75
    elif wickets == '4 wickets':
        points = 108
    elif wickets == '5 wickets':
        points = 149
    elif wickets == '6 wickets':
        points = 174
    elif wickets == '7 wickets':
        points = 199


    if 5 <= economy <= 6:
        points += 2  # 2 points if economy between 5 and 6
    elif 4 <= economy < 4.99:
        points += 4  # 4 points if economy between 4 and 4.99

    return points

# Apply the functions to create new columns for batting points and bowling points
prediction_df_all['Predicted Bowling Points'] = prediction_df_all.apply(calculate_bowling_points, axis=1)
prediction_df_all

,Match_id,PlayerName,Predicted Wickets,Predicted Economy,Actual Wickets,Actual Economy,Actual Bowling Points,Predicted Bowling Points
0,980901,I Sharma,0 wickets,8.097161,2,6.500000,50,0
1,980901,M Ashwin,0 wickets,8.189979,1,4.000000,29,0
2,980901,MR Marsh,0 wickets,8.124389,2,5.000000,52,0
3,980901,R Ashwin,0 wickets,7.434040,1,7.000000,25,0
4,980901,R Bhatia,0 wickets,7.778415,1,2.500000,25,0
...,...,...,...,...,...,...,...,...
5543,1370353,J Little,1 wickets,9.182677,0,15.000000,0,25
5544,1370353,MM Sharma,0 wickets,8.429571,3,12.000000,75,0
5545,1370353,Mohammed Shami,1 wickets,8.280243,0,9.666667,0,25
5546,1370353,Noor Ahmad,1 wickets,7.752436,2,4.666667,54,25


In [ ]:
prediction_df_all.columns

Index(['Match_id', 'PlayerName', 'Predicted Wickets', 'Predicted Economy',
       'Actual Wickets', 'Actual Economy', 'Actual Bowling Points',
       'Predicted Bowling Points'],
      dtype='object')

**SORTING PLAYERS BY HIGHEST POINTS MATCH WISE**

In [ ]:
# first sorting according to predicted points
bowling_points_data_sorted = prediction_df_all.sort_values(by=['Match_id', 'Predicted Bowling Points'], ascending=[True, False])
bowling_points_data_sorted.to_csv('bowling_players_predictions.csv', index=False)

In [ ]:
# now sorting according to actual points
bowling_points_data_sorted_actual = prediction_df_all.sort_values(by=['Match_id', 'Actual Bowling Points'], ascending=[True, False])
bowling_points_data_sorted_actual.to_csv('bowling_players_actual.csv', index=False)




# **COMBINING BATTING AND BOWLING POINTS**






In [ ]:
batting_data = pd.read_csv('batting_players_predictions.csv')
batting_data_actual = pd.read_csv('batting_players_actual.csv')
bowling_data = pd.read_csv('bowling_players_predictions.csv')
bowling_data_actual = pd.read_csv('bowling_players_actual.csv')

In [ ]:
batting_data.columns

Index(['Match_id', 'PlayerName', 'Batsman', 'Total_Runs_Predicted',
       'Fours_Predicted', 'Sixes_Predicted', 'balls_faced', 'Actual Runs',
       'Actual Fours', 'Actual Sixes', 'Actual Balls Faced', 'Actual SR', 'SR',
       'Predicted Batting Points', 'Actual Batting Points'],
      dtype='object')

In [ ]:
bowling_data.columns

Index(['Match_id', 'PlayerName', 'Predicted Wickets', 'Predicted Economy',
       'Actual Wickets', 'Actual Economy', 'Actual Bowling Points',
       'Predicted Bowling Points'],
      dtype='object')

In [ ]:
merge = pd.merge(batting_data,bowling_data, on = ["Match_id","PlayerName"], how = 'outer')
merge.fillna(0,inplace = True)

merge["Actual total points"] = merge["Actual Batting Points"] + merge["Actual Bowling Points"]
merge["Predicted total points"] = merge["Predicted Batting Points"] + merge["Predicted Bowling Points"]
merge.to_csv("total_points.csv", index = False)

merge

,Match_id,PlayerName,Batsman,Total_Runs_Predicted,Fours_Predicted,Sixes_Predicted,balls_faced,Actual Runs,Actual Fours,Actual Sixes,...,Predicted Batting Points,Actual Batting Points,Predicted Wickets,Predicted Economy,Actual Wickets,Actual Economy,Actual Bowling Points,Predicted Bowling Points,Actual total points,Predicted total points
0,980901,AM Rahane,0.0,43.0,6.0,0.0,33.0,65.0,7.0,3.0,...,49.0,80.0,0,0.000000,0.0,0.000000,0.0,0.0,80.0,49.0
1,980901,F du Plessis,0.0,41.0,5.0,0.0,31.0,34.0,1.0,3.0,...,46.0,35.0,0,0.000000,0.0,0.000000,0.0,0.0,35.0,46.0
2,980901,AT Rayudu,0.0,29.0,1.0,0.0,21.0,22.0,2.0,0.0,...,30.0,24.0,0,0.000000,0.0,0.000000,0.0,0.0,24.0,30.0
3,980901,Harbhajan Singh,0.0,23.0,1.0,0.0,18.0,45.0,7.0,1.0,...,24.0,52.0,0 wickets,8.378965,1.0,8.000000,25.0,0.0,77.0,24.0
4,980901,HH Pandya,0.0,20.0,3.0,0.0,15.0,9.0,2.0,0.0,...,23.0,11.0,0 wickets,8.697697,0.0,18.000000,0.0,0.0,11.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10671,1370353,Mohammed Shami,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1 wickets,8.280243,0.0,9.666667,0.0,25.0,0.0,25.0
10672,1370353,Noor Ahmad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1 wickets,7.752436,2.0,4.666667,54.0,25.0,54.0,25.0
10673,1370353,DL Chahar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0 wickets,8.152317,1.0,9.500000,25.0,0.0,25.0,0.0
10674,1370353,M Theekshana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0 wickets,7.660359,0.0,9.000000,0.0,0.0,0.0,0.0


In [ ]:
final_sorted = merge.sort_values(by=['Match_id', 'Predicted total points'], ascending=[True, False])
final_sorted

,Match_id,PlayerName,Batsman,Total_Runs_Predicted,Fours_Predicted,Sixes_Predicted,balls_faced,Actual Runs,Actual Fours,Actual Sixes,...,Predicted Batting Points,Actual Batting Points,Predicted Wickets,Predicted Economy,Actual Wickets,Actual Economy,Actual Bowling Points,Predicted Bowling Points,Actual total points,Predicted total points
0,980901,AM Rahane,0.0,43.0,6.0,0.0,33.0,65.0,7.0,3.0,...,49.0,80.0,0,0.000000,0.0,0.000000,0.0,0.0,80.0,49.0
1,980901,F du Plessis,0.0,41.0,5.0,0.0,31.0,34.0,1.0,3.0,...,46.0,35.0,0,0.000000,0.0,0.000000,0.0,0.0,35.0,46.0
2,980901,AT Rayudu,0.0,29.0,1.0,0.0,21.0,22.0,2.0,0.0,...,30.0,24.0,0,0.000000,0.0,0.000000,0.0,0.0,24.0,30.0
12,980901,MJ McClenaghan,0.0,4.0,0.0,0.0,3.0,2.0,0.0,0.0,...,4.0,2.0,1 wickets,8.257939,0.0,9.000000,0.0,25.0,2.0,29.0
7614,980901,RP Singh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1 wickets,8.882007,1.0,9.666667,25.0,25.0,25.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7611,1370353,RA Jadeja,0.0,9.0,0.0,0.0,6.0,15.0,1.0,1.0,...,9.0,16.0,0 wickets,7.726036,1.0,9.500000,25.0,0.0,41.0,9.0
7612,1370353,MS Dhoni,0.0,6.0,0.0,0.0,4.0,0.0,0.0,0.0,...,6.0,-2.0,0,0.000000,0.0,0.000000,0.0,0.0,-2.0,6.0
10673,1370353,DL Chahar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0 wickets,8.152317,1.0,9.500000,25.0,0.0,25.0,0.0
10674,1370353,M Theekshana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0 wickets,7.660359,0.0,9.000000,0.0,0.0,0.0,0.0


In [ ]:
final_sorted.to_csv('final_filtered.csv', index=False)

In [ ]:
actual_final_sorted = merge.sort_values(by=['Match_id', 'Actual total points'], ascending=[True, False])
actual_final_sorted.to_csv('actual_final_filtered.csv', index=False)